# 三目並べゲーム
参考:https://github.com/narisan25/TTT-RL
## ボードと環境

In [1]:
import pygame
import random

# POS State
EMPTY = 0  # 空の状態を表す定数
PLAYER_X = 1  # プレイヤーXを表す定数
PLAYER_O = -1  # プレイヤーOを表す定数
DRAW = 2  # 引き分けを表す定数

class TTTBoard:
    """ボードを管理するクラス"""
    def __init__(self, board=None):
        if board is None:
            self.board = [EMPTY] * 9  # ボードを初期化
        else:
            self.board = board
        self.winner = None  # 勝者を保持する変数
        self.invalid_act = None  # 無効な行動を保持する変数
    
    def get_possible_pos(self):
        """可能な位置を取得"""
        return [i for i, cell in enumerate(self.board) if cell == EMPTY]  # 空の位置をリストで返す
    
    def pygame_init(self):
        """pygame開始"""
        pygame.init()  # pygameを初期化
        self.screen = pygame.display.set_mode((300, 300))  # ウィンドウのサイズを設定
        self.font = pygame.font.Font(None, 100)  # フォントを設定
        pygame.display.set_caption("Tic Tac Toe")  # ウィンドウのタイトルを設定
        self.pygame_render(self.board)  # 初期描画
    
    def pygame_render(self, board):
        """pygame描画"""
        WHITE = (255, 255, 255)  # 白色
        BLACK = (0, 0, 0)  # 黒色
        
        self.screen.fill(WHITE)  # 画面を白で塗りつぶす
        
        for x in range(1, 3):
            pygame.draw.line(self.screen, BLACK, (x * 100, 0), (x * 100, 300), 3)  # 垂直線を描画
            pygame.draw.line(self.screen, BLACK, (0, x * 100), (300, x * 100), 3)  # 水平線を描画
            
        for i in range(9):
            x = i % 3
            y = i // 3
            if board[i] == PLAYER_X:
                text = self.font.render('X', True, BLACK)  # 'X'を描画
                self.screen.blit(text, (x * 100 + 25, y * 100 + 15))
            elif board[i] == PLAYER_O:
                text = self.font.render('O', True, BLACK)  # 'O'を描画
                self.screen.blit(text, (x * 100 + 25, y * 100 + 15))
        
        pygame.display.flip()  # 画面を更新

    def check_winner(self):
        """勝ちを確認"""
        win_cond = ((0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6))  # 勝利条件
        for each in win_cond:
            if self.board[each[0]] == self.board[each[1]] == self.board[each[2]] and self.board[each[0]] != EMPTY:
                self.winner = self.board[each[0]]  # 勝者を設定
                return self.winner
        return None
    
    def check_draw(self):
        """引き分けを確認"""
        if not any(cell == EMPTY for cell in self.board) and self.winner is None:
            self.winner = DRAW  # 引き分けを設定
            return DRAW
        return None
    
    def step(self, pos, player):
        """次の状態"""
        if self.board[pos] == EMPTY:
            self.board[pos] = player  # プレイヤーの位置を設定
            self.invalid_act = None
        else:
            self.invalid_act = pos  # 無効な位置を設定
            self.winner = -1 * player  # 相手の勝利を設定
        self.check_winner()  # 勝利の確認
        self.check_draw()  # 引き分けの確認
    
    def clone(self):
        return TTTBoard(self.board.copy())  # ボードのコピーを返す

    def switch_player(self):
        if self.player_turn == PLAYER_X:
            self.player_turn = PLAYER_O  # プレイヤーを切り替える
        else:
            self.player_turn = PLAYER_X  # プレイヤーを切り替える

pygame 2.5.2 (SDL 2.28.3, Python 3.10.14)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class TTTenv:

    act_turn = 0  # アクションターンの初期化
    winner = None  # 勝者の初期化
    
    def __init__(self, px, po, nplay=-1, showBoard=True, showResult=True, stat=100):
        self.player_x = px  # プレイヤーXを設定
        self.player_o = po  # プレイヤーOを設定
        self.nwon = {px.myturn: 0, po.myturn: 0, DRAW: 0}  # 勝利数の初期化
        self.nplay = nplay  # プレイ回数の設定
        self.players = (self.player_x, self.player_o)  # プレイヤーのタプル
        self.board = None  # ボードの初期化
        self.disp = showBoard  # ボード表示の設定
        self.showResult = showResult  # 結果表示の設定
        self.player_turn = self.players[random.randrange(2)]  # ランダムにプレイヤーのターンを決定
        self.nplayed = 0  # プレイ済みの回数
        self.stat = stat  # 統計の表示間隔
    
    def progress(self, random_turn=True):
        """ゲームの進行"""
        while self.nplayed != self.nplay:
            if random_turn:
                self.player_turn = self.players[random.randrange(2)]  # ランダムにプレイヤーのターンを決定
            self.board = TTTBoard()  # 新しいボードを作成
            if self.disp:
                self.board.pygame_init()  # pygameの初期化と表示
            while self.board.winner is None:
                act = self.player_turn.act(self.board)  # プレイヤーの行動を取得
                self.board.step(act, self.player_turn.myturn)  # 行動をボードに反映
                if self.disp: self.board.pygame_render(self.board.board)  # ボードを描画
               
                if self.board.winner is not None:
                    for i in self.players:
                        i.getGameResult(self.board)  # 各プレイヤーに結果を通知
                    if self.board.winner == DRAW:
                        if self.showResult: print("Draw Game")  # 引き分けの場合の表示
                    elif self.board.winner == self.player_turn.myturn:
                        out = "Winner : " + self.player_turn.name  # 勝者の表示
                        if self.showResult: print(out)
                    else:
                        print("Invalid Act!")  # 無効な行動の表示
                    self.nwon[self.board.winner] += 1  # 勝利数を更新
                else:
                    self.switch_player()  # プレイヤーのターンを切り替える
                    self.player_turn.getGameResult(self.board)  # 現在のボードの結果を取得

            self.nplayed += 1  # プレイ済みの回数を更新
            if self.nplayed % self.stat == 0 or self.nplayed == self.nplay:
                print(f"{self.player_x.name}:{self.nwon[self.player_x.myturn]},\
                {self.player_o.name}:{self.nwon[self.player_o.myturn]},\
                DRAW:{self.nwon[DRAW]}")  # 統計の表示
                if self.disp: pygame.quit()  # pygameを終了
                    
    def switch_player(self):
        """プレイヤーのターンを切り替える"""
        if self.player_turn == self.player_x:
            self.player_turn = self.player_o  # プレイヤーOに切り替え
        else:
            self.player_turn = self.player_x  # プレイヤーXに切り替え

## ランダムとランダムαと人間

In [3]:
import sys
class RandomAgent:
    """ただのランダム"""
    def __init__(self, turn):
        self.name = "Random"
        self.myturn = turn
        
    def act(self, board):
        acts = board.get_possible_pos()
        i = random.randrange(len(acts))
        return acts[i]
    
    def getGameResult(self, board):
        pass

class AlphaRandomAgent:
    """勝てるところがあれば勝ちに行くランダム"""
    
    def __init__(self,turn,name="AlphaRandom"):
        self.name=name
        self.myturn=turn
        
    def getGameResult(self,winner):
        pass
        
    def act(self,board):
        acts=board.get_possible_pos()
        #see only next winnable act
        for act in acts:
            tempboard=board.clone()
            tempboard.step(act,self.myturn)
            # check if win
            if tempboard.winner==self.myturn:
                #print ("Check mate")
                return act
        i=random.randrange(len(acts))
        return acts[i]


class HumanAgent:
    """人が操作する"""
    def __init__(self, turn):
        self.name = "Human"
        self.myturn = turn
        
    def act(self, board):
        valid = False
        while not valid:
            for event in pygame.event.get():# Pygameのイベントを処理する
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:# マウスの位置を取得する
                    
                    x, y = event.pos # マウスの位置をボードのセルに変換する
                    row = y // 100
                    col = x // 100
                    act = row * 3 + col
                    if act >= 0 and act < 9 and board.board[act] == EMPTY:
                        valid = True
                        return act
                    else:
                        board.pygame_render(board.board)
                        RED = (255, 0, 0, 150)  # 赤色（透明度150）    
                        pygame.draw.rect(board.screen, RED, (col * 100, row * 100, 100, 100), 3)  # 無効な位置を赤で囲む
                        pygame.display.flip()  # 画面を更新することで描画が反映される
    
    def getGameResult(self, board):
        pass

In [4]:
def vsAlphaRandom(): 
    p1=HumanAgent(PLAYER_X)
    p2=AlphaRandomAgent(PLAYER_O)
    game=TTTenv(p1,p2)
    game.progress()
#vsAlphaRandom()

## モンテカルロ法

In [5]:
class MCAgent:
    def __init__(self,turn,name="MC"):
        self.name=name
        self.myturn=turn
    
    def getGameResult(self,winner):
        pass
        
    def win_or_rand(self,board,turn):
        acts=board.get_possible_pos()
        #see only next winnable act
        for act in acts:
            tempboard=board.clone()
            tempboard.step(act,turn)
            # check if win
            if tempboard.winner==turn:
                return act
        i=random.randrange(len(acts))
        return acts[i]
           
    def trial(self,score,board,act):
        tempboard=board.clone()
        tempboard.step(act,self.myturn)
        tempturn=self.myturn
        while tempboard.winner is None:
            tempturn=tempturn*-1
            tempboard.step(self.win_or_rand(tempboard,tempturn),tempturn)
        
        if tempboard.winner==self.myturn:
            score[act]+=1
        elif tempboard.winner==DRAW:
            pass
        else:
            score[act]-=1

        
    def getGameResult(self,board):
        pass
        
    
    def act(self,board):
        acts=board.get_possible_pos()
        scores={}
        n=50
        for act in acts:
            scores[act]=0
            for i in range(n):
                #print("Try"+str(i))
                self.trial(scores,board,act)
            
            #print(scores)
            scores[act]/=n
        
        max_score=max(scores.values())
        for act, v in scores.items():
            if v == max_score:
                #print(str(act)+"="+str(v))
                return act

In [6]:
def vsMC():
    p1=HumanAgent(PLAYER_X)
    p2=MCAgent(PLAYER_O,"M2")
    game=TTTenv(p1,p2)
    game.progress()
#vsMC()

## Q-Learning

In [7]:
import pickle

class QLAgent:
    def __init__(self, turn, name="QL", epsilon=0.2, alpha=0.3):
        self.name = name  # エージェントの名前
        self.myturn = turn  # エージェントのターン（PLAYER_X or PLAYER_O）
        self.q = {}  # Qテーブル (state, action) -> Q値 のマッピング
        self.epsilon = epsilon  # ε-greedy の ε パラメータ (探索の確率)
        self.alpha = alpha  # 学習率
        self.gamma = 0.9  # 割引率
        self.last_move = None  # 前回の行動
        self.last_board = None  # 前回のボードの状態
        self.totalgamecount = 0  # 総ゲーム数
        
    def act(self, board):
        """エージェントの行動を決定する"""
        self.last_board = board.clone()  # 現在のボード状態を保存
        acts = board.get_possible_pos()  # 可能な行動を取得

        # ε-greedy 探索：ランダムに行動を選択
        if random.random() < (self.epsilon / (self.totalgamecount // 10000 + 1)):
            i = random.randrange(len(acts))
            return acts[i]

        # Q値に基づいて行動を選択
        qs = [self.getQ(tuple(self.last_board.board), act) for act in acts]
        maxQ = max(qs)

        if qs.count(maxQ) > 1:
            # 最良の選択肢が複数ある場合、それらからランダムに選択
            best_options = [i for i in range(len(acts)) if qs[i] == maxQ]
            i = random.choice(best_options)
        else:
            i = qs.index(maxQ)

        self.last_move = acts[i]
        return acts[i]
    
    def getQ(self, state, act):
        """Q値を取得する"""
        if self.q.get((state, act)) is None:
            self.q[(state, act)] = 1  # 初期Q値を1とする
        return self.q.get((state, act))
    
    def getGameResult(self, board):
        """ゲーム結果を処理する"""
        r = 0
        if self.last_move is not None:
            if board.winner is None:
                self.learn(self.last_board, self.last_move, 0, board)
            else:
                if board.winner == self.myturn:
                    self.learn(self.last_board, self.last_move, 1, board)
                elif board.winner != DRAW:
                    self.learn(self.last_board, self.last_move, -1, board)
                else:
                    self.learn(self.last_board, self.last_move, 0, board)
                self.totalgamecount += 1
                self.last_move = None
                self.last_board = None

    def learn(self, s, a, r, fs):
        """Q学習の更新を行う"""
        pQ = self.getQ(tuple(s.board), a)
        if fs.winner is not None:
            maxQnew = 0
        else:
            maxQnew = max([self.getQ(tuple(fs.board), act) for act in fs.get_possible_pos()])
        self.q[(tuple(s.board), a)] = pQ + self.alpha * ((r + self.gamma * maxQnew) - pQ)

    def save_weights(self, filepath='agt_data/noname'):
        """Qテーブルをファイルに保存する"""
        filepath = filepath + '.pkl'
        with open(filepath, mode='wb') as f:
            pickle.dump(self.q, f)

    def load_weights(self, filepath='agt_data/noname'):
        """ファイルからQテーブルを読み込む"""
        filepath = filepath + '.pkl'
        with open(filepath, mode='rb') as f:
            self.q = pickle.load(f)

In [8]:
def trainQL():
    p1=QLAgent(PLAYER_O,"QL1")
    p2=QLAgent(PLAYER_X,"QL2")
    game=TTTenv(p1,p2,100000,False,False,10000)
    game.progress()
    p1.save_weights(filepath='agt_data/tictactoe_QL')
#trainQL()

In [9]:
def vsQL():
    p1=QLAgent(PLAYER_O,"QL1")
    p1.epsilon=0
    p2=HumanAgent(PLAYER_X)
    p1.load_weights(filepath='agt_data/tictactoe_QL')
    game=TTTenv(p1,p2)
    game.player_turn = game.players[1]# 先攻
    game.progress(random_turn=False)
vsQL()

SystemExit: 

C:\Users\msy-t\anaconda3\envs\DL\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# DQN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from collections import deque
import random
import numpy as np

class DQNAgent:
    def __init__(self, turn, name="DQN",
                 gamma = 0.95, # 割引率
                 epsilon = 1, # 乱雑度
                ):
        
        # パラメータ
        self.name = name  # エージェントの名前
        self.myturn = turn  # エージェントのターン（PLAYER_X or PLAYER_O）
        self.input_size = (9,)
        self.n_act = 9
        self.gamma = gamma 
        self.epsilon = epsilon  
        self.model = self._build_Qnet()
        self.totalgamecount = 0
        self.last_move = None
        self.last_board = None
        self.last_pred = None
        # 報酬パラメータ
        self.rwin, self.rlose, self.rdraw, self.rmiss = 1, -1, 0, -1.5
                
    def _build_Qnet(self):
        """Qネットワークの構築"""
        model = Sequential()
        model.add(Flatten(input_shape=self.input_size))
        model.add(Dense(162, activation='relu'))
        model.add(Dense(162, activation='relu'))
        model.add(Dense(self.n_act, activation='linear'))
        
        # 勾配法のパラメータの定義
        model.compile(loss='mse', optimizer='Adam')
        
        return model

    def act(self, board):
        """行動を決定"""
        self.last_board = board.clone()  # 現在のボード状態を保存。clone() メソッドを使ってオブジェクトをコピー。
        x = np.array([board.board], dtype=np.float32)
        
        pred = self.model.predict(x, verbose=0)[0, :]  # モデルに入力データを渡し、予測値を取得。
        self.last_pred = pred
        act = np.argmax(pred)  # 予測値の中で最大値を持つインデックスを取得し、行動を決定。
        
        if self.epsilon > 0.2:  # εを時間経過とともに減少させる（ε-greedy アルゴリズム）。
            self.epsilon -= 1 / 20000
        if random.random() < self.epsilon:  # εの確率でランダムな行動を選択。
            acts = board.get_possible_pos()  # 可能な行動を取得。
            i = random.randrange(len(acts))  # ランダムに行動を選択。
            act = acts[i]
        i = 0
        while board.board[act] != EMPTY:  # 無効な行動の場合、繰り返し有効な行動を見つけるまでループ。
            self.learn(self.last_board, act, -1, self.last_board)  # 無効な行動の場合、報酬-1として学習。
            x = np.array([board.board], dtype=np.float32)
            pred = self.model.predict(x, verbose=0)[0, :]  # モデルに入力データを渡し、再度予測値を取得。
            act = np.argmax(pred)
            i += 1
            if i > 10:  # 10回以上繰り返しても有効な行動が見つからない場合の処理。
                print("Exceed Pos Find" + str(board.board) + " with " + str(act))
                acts = self.last_board.get_possible_pos()  # 前回のボード状態から可能な行動を取得。
                act = acts[random.randrange(len(acts))]

        self.last_move = act  # 最終的に選択された行動を保存。
        return act  # 選択された行動を返す。
            
    def getGameResult(self, board):
        """ゲームの結果を処理"""
        r = 0
        if self.last_move is not None:  # 前の行動が存在する場合にのみ処理を行う
            if board.winner is None:  # 勝者が決まっていない場合
                self.learn(self.last_board, self.last_move, 0, board)  # 報酬0で学習
            else:
                if board.board == self.last_board.board:  # ボード状態が前回と同じ場合
                    self.learn(self.last_board, self.last_move, self.rmiss, board)  # 誤った行動として学習
                elif board.winner == self.myturn:  # 自分が勝者の場合
                    self.learn(self.last_board, self.last_move, self.rwin, board)  # 勝利の報酬で学習
                elif board.winner != DRAW:  # 相手が勝者の場合
                    self.learn(self.last_board, self.last_move, self.rlose, board)  # 敗北の報酬で学習
                else:  # 引き分けの場合
                    self.learn(self.last_board, self.last_move, self.rdraw, board)  # 引き分けの報酬で学習
                self.totalgamecount += 1  # 総ゲーム数をインクリメント
                self.last_move = None  # 前回の行動をリセット
                self.last_board = None  # 前回のボード状態をリセット
                self.last_pred = None  # 前回の予測結果をリセット
             
    def learn(self, obs, act, rwd, next_obs):
        """学習"""
        # ゲームの次の状態(next_obs)の勝者が存在するかをチェック
        if next_obs.winner is not None:
            maxQnew = 0  # 勝者が存在する場合、次の状態の最大Q値は0とする
        else:
            # モデルに次の状態を入力し、予測値の最大値を取得
            x = np.array([next_obs.board], dtype=np.float32)
            Q = self.model.predict(x, verbose=0)[0, :]
            maxQnew = np.max(Q)
        
        # Q値の更新式に基づいて、更新するQ値を計算
        update = rwd + self.gamma * maxQnew

        # 前回の行動に対するQ値を更新
        self.last_pred[act] = update
        
        x = np.array([obs.board], dtype=np.float32)
        t = np.array([self.last_pred], dtype=np.float32)
        
        self.model.fit(x, t, verbose=0, epochs=1)     
    
    def save_weights(self, filepath='agt_data/noname'):
        """モデルを保存する"""
        self.model.save(filepath + '.keras', overwrite=True)

    def load_weights(self, filepath='agt_data/noname'):
        """モデルの重みを読み込む"""
        self.model = tf.keras.models.load_model(filepath + '.keras')


In [ ]:
pDQ=DQNAgent(PLAYER_X)
p2=AlphaRandomAgent(PLAYER_O)
game=TTTenv(pDQ,p2,1000,False,False,100)
game.progress()

In [ ]:
pDQ.e=1
pQ=QLAgent(PLAYER_O,"QL1")
pQ.epsilon=0
pQ.load_weights(filepath='agt_data/tictactoe_QL')
game=TTTenv(pDQ,pQ,30000,False,False,1000)
game.progress()

In [7]:
import numpy as np

# リストを定義
data = [[0,0,0],[0,0,0],[0,0,0]]

# NumPy配列に変換
data_array = np.array(data)
print(data_array)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [8]:
# 現在の形状を確認
print(data_array.shape)


(3, 3)


In [9]:
import tensorflow as tf

# NumPy配列をテンソルに変換
data_tensor = tf.convert_to_tensor(data_array, dtype=tf.float32)
print(data_tensor)


tf.Tensor(
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]], shape=(3, 3), dtype=float32)


In [10]:
# バッチ次元を追加
data_tensor = tf.expand_dims(data_tensor, axis=0)
print(data_tensor)


tf.Tensor(
[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]], shape=(1, 3, 3), dtype=float32)


In [11]:
# 簡単なモデルの定義
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(3, 3)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# モデルのコンパイル
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# モデルへの入力
output = model(data_tensor)
print(output)


tf.Tensor([[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]], shape=(1, 10), dtype=float32)


C:\Users\msy-t\anaconda3\envs\DL\lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
